In [22]:
from transformers import ASTConfig, ASTForAudioClassification

config = ASTConfig.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")

In [11]:
config.name_or_path

''

In [23]:
config.to_json_file("config.json")

In [24]:
import wandb
import sys
sys.path.append("..")
from settings import PROJECT, ENTITY, JobType, ArtifactType

with wandb.init(
        project=PROJECT, entity=ENTITY, job_type=JobType.UPLOAD_CONFIG.value
) as run:
    artifact = wandb.Artifact(
        "ast-feature-extractor-config",
        type=ArtifactType.CONFIG_FILE.value,
        description="AST feature extractor config file",
    )
    artifact.add_file("config.json", "config.json")
    
    run.log_artifact(artifact)
    

wandb: Currently logged in as: simonexc (dl-mini). Use `wandb login --relogin` to force relogin


In [30]:
from transformers import ASTConfig, ASTForAudioClassification, ASTFeatureExtractor
import sys
sys.path.append("..")
from settings import NUM_CLASSES
config = ASTConfig.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")
config.num_labels = NUM_CLASSES
config.max_length = 100
feature_extractor = ASTFeatureExtractor.from_dict(config.to_dict())

In [25]:
from dataset.training_dataset import SpeechDataset

dataset = SpeechDataset("/home/simonexc/Documents/GitHub/MINI_DL_RNN/src/artifacts/speech-waveform:v0", 10, None, None)

In [26]:
dataset.setup()
x = dataset.train.tensors[0]
x

tensor([[ 5.1880e-04,  1.1597e-03,  1.3123e-03,  ...,  7.9346e-04,
          1.1902e-03,  1.4954e-03],
        [ 1.8646e-02,  2.4384e-02,  2.8320e-02,  ...,  3.0670e-02,
          2.8564e-02,  2.3407e-02],
        [-5.0659e-03, -6.9885e-03, -8.8196e-03,  ...,  1.0620e-02,
          7.9041e-03,  7.0801e-03],
        ...,
        [ 7.5684e-03,  8.6365e-03,  7.4768e-03,  ...,  1.1780e-02,
          1.2634e-02,  1.0040e-02],
        [ 1.8311e-04,  2.7466e-04,  2.1362e-04,  ...,  7.9346e-04,
          7.0190e-04,  4.5776e-04],
        [ 9.1553e-05,  1.2207e-04, -1.2207e-04,  ...,  0.0000e+00,
          1.2207e-04, -1.8311e-04]])

In [31]:
a = feature_extractor(x.numpy()[:1], sampling_rate=16000, return_tensors="pt").input_values

In [38]:
import numpy as np
d = np.empty((0, 100, 128))
for xs, _ in dataset.train_dataloader():
    xs = feature_extractor(xs.numpy(), sampling_rate=16000, return_tensors="pt").input_values.numpy()
    d = np.concatenate((d, xs))

In [40]:
print(d.mean(), d.std())

-0.4722489706457146 0.5442722093877054


In [43]:
e = (d-d.mean()) / (2*d.std())

In [44]:
e.mean(), e.std()

(5.4055588297990365e-15, 0.49999999999999706)

In [32]:
a.shape

torch.Size([1, 100, 128])

In [27]:
from transformers.models.audio_spectrogram_transformer.modeling_audio_spectrogram_transformer import ASTEmbeddings
config = ASTConfig.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")
config.max_length = 100
embedding = ASTEmbeddings(config)

In [28]:
e = embedding(a)

In [29]:
e.shape

torch.Size([1, 110, 768])